In [ ]:
import tkinter as tk
from tkinter import filedialog, ttk
import segyio
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

class SeismicCurtainApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Seismic Curtain Effect")
        self.root.geometry("1000x700")

        # Variables
        self.cube1 = None
        self.cube2 = None
        self.view_type = tk.StringVar(value="inline")
        self.slice_num = tk.IntVar(value=0)
        self.curtain_pos = tk.DoubleVar(value=0.5)
        self.cube1_intensity = tk.DoubleVar(value=1.0)  # Intensity scale for cube1
        self.cube2_intensity = tk.DoubleVar(value=1.0)  # Intensity scale for cube2
        self.current_slice1 = None
        self.current_slice2 = None
        self.image = None
        self.curtain_line = None
        self.curtain_icon = None

        # GUI Setup
        self.setup_gui()

        # Matplotlib setup
        self.fig, self.ax = plt.subplots(figsize=(8, 6))
        self.canvas = FigureCanvasTkAgg(self.fig, master=self.root)
        self.canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=1)
        
        # Bind mouse events for dragging
        self.canvas.mpl_connect('button_press_event', self.on_press)
        self.canvas.mpl_connect('motion_notify_event', self.on_drag)
        self.canvas.mpl_connect('button_release_event', self.on_release)
        self.dragging = False

        # Initial empty plot
        self.update_plot()

    def setup_gui(self):
        control_frame = ttk.Frame(self.root)
        control_frame.pack(side=tk.BOTTOM, fill=tk.X, padx=5, pady=5)

        ttk.Button(control_frame, text="Upload Bottom Cube", command=self.load_cube1).pack(side=tk.LEFT, padx=5)
        ttk.Button(control_frame, text="Upload Top Cube", command=self.load_cube2).pack(side=tk.LEFT, padx=5)

        ttk.Label(control_frame, text="View Type:").pack(side=tk.LEFT, padx=5)
        for vt in ["inline", "crossline", "time slice"]:
            ttk.Radiobutton(control_frame, text=vt, value=vt, variable=self.view_type, 
                          command=self.update_plot).pack(side=tk.LEFT)

        ttk.Label(control_frame, text="Slice:").pack(side=tk.LEFT, padx=5)
        self.slice_slider = ttk.Scale(control_frame, from_=0, to=100, orient=tk.HORIZONTAL, 
                                    variable=self.slice_num, command=self.update_slice)
        self.slice_slider.pack(side=tk.LEFT, padx=5)

        ttk.Label(control_frame, text="Curtain Pos:").pack(side=tk.LEFT, padx=5)
        self.curtain_slider = ttk.Scale(control_frame, from_=0, to=1, orient=tk.HORIZONTAL, 
                                      variable=self.curtain_pos, command=self.update_curtain)
        self.curtain_slider.pack(side=tk.LEFT, padx=5)

        # Intensity controls
        ttk.Label(control_frame, text="Bottom Intensity:").pack(side=tk.LEFT, padx=5)
        self.cube1_intensity_slider = ttk.Scale(control_frame, from_=0.1, to=2.0, orient=tk.HORIZONTAL, 
                                              variable=self.cube1_intensity, command=self.update_curtain)
        self.cube1_intensity_slider.pack(side=tk.LEFT, padx=5)

        ttk.Label(control_frame, text="Top Intensity:").pack(side=tk.LEFT, padx=5)
        self.cube2_intensity_slider = ttk.Scale(control_frame, from_=0.1, to=2.0, orient=tk.HORIZONTAL, 
                                              variable=self.cube2_intensity, command=self.update_curtain)
        self.cube2_intensity_slider.pack(side=tk.LEFT, padx=5)

    def load_cube1(self):
        file_path = filedialog.askopenfilename(filetypes=[("SEG-Y files", "*.segy")])
        if file_path:
            self.cube1 = segyio.open(file_path, mode='r')
            self.update_slider_range()
            self.update_plot()

    def load_cube2(self):
        file_path = filedialog.askopenfilename(filetypes=[("SEG-Y files", "*.segy")])
        if file_path:
            self.cube2 = segyio.open(file_path, mode='r')
            self.update_slider_range()
            self.update_plot()

    def update_slider_range(self):
        if self.cube1:
            if self.view_type.get() == "inline":
                self.slice_slider.configure(to=len(self.cube1.ilines)-1)
            elif self.view_type.get() == "crossline":
                self.slice_slider.configure(to=len(self.cube1.xlines)-1)
            else:  # time slice
                self.slice_slider.configure(to=len(self.cube1.samples)-1)

    def update_slice(self, *args):
        self.update_plot()

    def get_slice(self, cube, view_type, slice_num):
        if not cube:
            return np.zeros((100, 100))
        if view_type == "inline":
            return cube.iline[cube.ilines[slice_num]]  # Shape: (time, xline)
        elif view_type == "crossline":
            return cube.xline[cube.xlines[slice_num]]  # Shape: (time, inline)
        else:  # time slice
            return cube.depth_slice[slice_num]  # Shape: (inline, xline)

    def on_press(self, event):
        if event.inaxes != self.ax or self.curtain_line is None:
            return
        curtain_x = self.curtain_line.get_xdata()[0]
        if abs(event.xdata - curtain_x) < 10:
            self.dragging = True

    def on_drag(self, event):
        if self.dragging and event.inaxes == self.ax:
            x_min, x_max = self.ax.get_xlim()
            new_pos = (event.xdata - x_min) / (x_max - x_min)
            self.curtain_pos.set(max(0, min(1, new_pos)))
            self.update_curtain()

    def on_release(self, event):
        self.dragging = False

    def update_plot(self, *args):
        self.ax.clear()
        
        # Get and transpose data slices
        slice1 = self.get_slice(self.cube1, self.view_type.get(), int(self.slice_num.get())).T
        slice2 = self.get_slice(self.cube2, self.view_type.get(), int(self.slice_num.get())).T

        # Ensure compatible shapes
        if slice1.shape != slice2.shape:
            min_shape = (min(slice1.shape[0], slice2.shape[0]), 
                        min(slice1.shape[1], slice2.shape[1]))
            slice1 = slice1[:min_shape[0], :min_shape[1]]
            slice2 = slice2[:min_shape[0], :min_shape[1]]

        # Store slices for quick updates
        self.current_slice1 = slice1
        self.current_slice2 = slice2

        # Default extent
        extent = [slice1.shape[1], 0, slice1.shape[0], 0]
        xlabel, ylabel = "X", "Y"

        # Set extent and labels if cube1 is loaded
        view_type = self.view_type.get()
        if self.cube1:
            if view_type == "inline":
                extent = [self.cube1.xlines[-1], self.cube1.xlines[0], 
                          self.cube1.samples[-1], self.cube1.samples[0]]
                xlabel, ylabel = "Crossline", "Time/Depth"
            elif view_type == "crossline":
                extent = [self.cube1.ilines[-1], self.cube1.ilines[0], 
                          self.cube1.samples[-1], self.cube1.samples[0]]
                xlabel, ylabel = "Inline", "Time/Depth"
            else:  # time slice
                extent = [self.cube1.xlines[-1], self.cube1.xlines[0], 
                          self.cube1.ilines[-1], self.cube1.ilines[0]]
                xlabel, ylabel = "Crossline", "Inline"

        # Initial plot (will be updated by update_curtain)
        self.image = self.ax.imshow(slice1, cmap='seismic', aspect='auto', extent=extent)
        self.ax.set_xlabel(xlabel)
        self.ax.set_ylabel(ylabel)
        self.ax.set_title(f"{view_type} Slice {int(self.slice_num.get())}")
        
        # Add curtain elements with intensity
        self.update_curtain()

    def update_curtain(self, *args):
        if self.current_slice1 is None or self.current_slice2 is None:
            return

        # Apply intensity scaling to each slice
        slice1_scaled = self.current_slice1 * self.cube1_intensity.get()
        slice2_scaled = self.current_slice2 * self.cube2_intensity.get()

        # Compute dynamic vmin/vmax based on scaled data
        vmin1, vmax1 = np.min(slice1_scaled), np.max(slice1_scaled)
        vmin2, vmax2 = np.min(slice2_scaled), np.max(slice2_scaled)
        vmin, vmax = min(vmin1, vmin2), max(vmax1, vmax2)

        # Create combined image with intensity applied
        curtain_x = int(self.curtain_pos.get() * self.current_slice1.shape[1])
        combined = np.copy(slice1_scaled)
        combined[:, :curtain_x] = slice2_scaled[:, :curtain_x]

        # Update image data with custom vmin/vmax
        if self.image:
            self.image.set_data(combined)
            self.image.set_clim(vmin, vmax)  # Set color limits based on combined range

        # Remove old curtain line and icon
        if self.curtain_line is not None:
            self.curtain_line.remove()
            self.curtain_line = None
        if self.curtain_icon is not None:
            self.curtain_icon.remove()
            self.curtain_icon = None

        # Add new curtain line and icon
        extent = self.image.get_extent()
        curtain_x_physical = extent[0] - (extent[0] - extent[1]) * self.curtain_pos.get()
        self.curtain_line = self.ax.axvline(x=curtain_x_physical, color='yellow', lw=2, ls='--')
        y_center = (extent[2] + extent[3]) / 2
        self.curtain_icon, = self.ax.plot(curtain_x_physical, y_center, 'y^', ms=20)

        self.canvas.draw_idle()

def main():
    root = tk.Tk()
    app = SeismicCurtainApp(root)
    root.mainloop()

if __name__ == "__main__":
    main()